In [1]:
import pygmt
import numpy as np
import pandas as pd
import geopy.distance

In [2]:
colombia_stations = 'Colombia-Noise.csv'
ecuador_stations = 'Ecuador-Noise.csv'
earthquakes_file = 'Earthquakes.csv'
cities_file = 'Cities.csv'

In [3]:
# Read in station locations and noise for each station
dfE = pd.read_csv(ecuador_stations)
dfC = pd.read_csv(colombia_stations)
frames = [dfC, dfE ]
df = pd.concat(frames)

In [4]:
cities = pd.read_csv(cities_file)

In [5]:
eqs = pd.read_csv(earthquakes_file)
eqs['Date'] = pd.to_datetime(eqs['Date'])

In [6]:
#minimum number of stations for detection
min = 4
#Seimic velocity in km/sec
vel = 4
# Time required to process and issue warning
issue = 6

In [7]:
cities

,Latitude,Longitude,City
0,4.7110,-74.0721,Bogota
1,6.2476,-75.5658,Medellin
2,3.4516,-76.5320,Cali
3,11.0041,-74.8070,Barranquilla
4,10.3932,-75.4832,Cartagena


In [8]:
for i in cities.itertuples():
    eqs[i[3]+'_(d)'] = eqs.apply(lambda row: geopy.distance.geodesic( [i[1], i[2]],(row.Latitude,row.Longitude)).km, axis=1)
eqs       

,Latitude,Longitude,Depth,Date,Mw,Bogota_(d),Medellin_(d),Cali_(d),Barranquilla_(d),Cartagena_(d)
0,4.800,-77.180,19.1,1991-11-19,7.2,344.926904,240.034517,165.556988,734.304366,646.272064
1,4.720,-77.570,16.0,2004-11-15,7.2,388.076099,279.032038,181.539524,758.793228,668.320425
2,2.849,-74.798,55.0,2067-02-09,7.0,221.122487,385.357127,203.930822,901.891370,837.746081
3,0.955,-79.369,25.0,2006-01-31,8.8,720.565270,721.756729,419.270076,1220.443297,1128.838142
4,0.025,-79.955,25.0,2042-05-14,7.8,834.503536,843.357703,537.200829,1341.101194,1248.895466
5,1.000,-79.400,60.0,2058-01-19,7.8,720.531318,719.754710,418.629159,1217.333828,1125.550565
6,0.380,-79.950,20.0,2016-04-16,7.7,810.217131,811.279789,509.838994,1305.377431,1212.692305
7,2.320,-78.810,19.7,1979-12-12,8.1,589.064050,564.164332,282.487221,1057.292145,965.589259


In [83]:
index=1
eq_loc = [eqs.iloc[index-1,0], eqs.iloc[index-1,1]]
cindex=3
city_loc = [cities.iloc[cindex-1,0], cities.iloc[cindex-1,1]]


In [84]:
df['distance']   = df.apply(lambda row: geopy.distance.geodesic( eq_loc,(row.lat,row.lon)).km, axis=1)
df.sort_values(by='distance', inplace=True, ignore_index=True)

In [85]:
city_dist = geopy.distance.geodesic(eq_loc,city_loc).km 
dect_dist = df.loc[min-1,'distance']
warning_time_sec = (city_dist - dect_dist)/vel